# Data Management básico con datatable

Realizaremos las mismas tareas que vimos en el anterior capítulo:

* Seleccionar columnas
* Eliminar columnas
* Seleccionar registros
* Crear nuevas variables
* Sumarizar datos
* Ordenar datos

Se va a emplear un conjunto de datos cargado en el repositorio del curso:

In [1]:
import datatable as dt
path = 'https://raw.githubusercontent.com/analisisydecision/intro_python_data_science/master/'
dt_df = dt.fread(path + 'index.csv')
dt_df.head()

,Creditability,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,1,1,18,4,2,1049,1,2,4,2,…,1,3,1,1,1
1,1,1,9,4,0,2799,1,3,2,3,…,2,3,2,1,1
2,1,2,12,2,9,841,2,4,2,2,…,1,2,1,1,1
3,1,1,12,4,0,2122,1,3,3,3,…,2,2,2,1,2
4,1,1,12,4,0,2171,1,3,4,3,…,2,2,1,1,2
5,1,1,10,4,0,2241,1,2,1,3,…,2,2,2,1,2
6,1,1,8,4,0,3398,1,4,1,3,…,2,2,1,1,2
7,1,1,6,4,0,1361,1,2,2,3,…,1,2,2,1,2
8,1,4,18,4,3,1098,1,1,4,2,…,2,1,1,1,1


El objeto generado será un data table frame, no es exactamente un data frame, sin embargo emplearemos esa definición. Tamaño del data frame cargado:

In [2]:
dt_df.shape

(1000, 21)

Nombre y tipo de las variables:

In [3]:
print(dt_df.names) 
print(dt_df.stypes)

('Creditability', 'Account Balance', 'Duration of Credit (month)', 'Payment Status of Previous Credit', 'Purpose', 'Credit Amount', 'Value Savings/Stocks', 'Length of current employment', 'Instalment per cent', 'Sex & Marital Status', 'Guarantors', 'Duration in Current address', 'Most valuable available asset', 'Age (years)', 'Concurrent Credits', 'Type of apartment', 'No of Credits at this Bank', 'Occupation', 'No of dependents', 'Telephone', 'Foreign Worker')
(stype.bool8, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32, stype.int32)


## Seleccionar columnas

Un objeto datatable tiene múltiples características, en este caso sólo queremos trabajar con columnas, el resto :

In [4]:
df2 = dt_df[:,['Occupation','No of dependents']]
df2.head(5)

,Occupation,No of dependents
,▪▪▪▪,▪▪▪▪
0,3,1
1,3,2
2,2,1
3,2,2
4,2,1


In [5]:
seleccionadas =['Occupation','No of dependents']
df2 = dt_df[:, seleccionadas]
df2.head(5)

,Occupation,No of dependents
,▪▪▪▪,▪▪▪▪
0,3,1
1,3,2
2,2,1
3,2,2
4,2,1


## Eliminar columnas

In [6]:
elimina = ['Creditability','Account Balance']
del dt_df[:, elimina]
dt_df.head()

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,18,4,2,1049,1,2,4,2,1,4,…,1,3,1,1,1
1,9,4,0,2799,1,3,2,3,1,2,…,2,3,2,1,1
2,12,2,9,841,2,4,2,2,1,4,…,1,2,1,1,1
3,12,4,0,2122,1,3,3,3,1,2,…,2,2,2,1,2
4,12,4,0,2171,1,3,4,3,1,4,…,2,2,1,1,2
5,10,4,0,2241,1,2,1,3,1,3,…,2,2,2,1,2
6,8,4,0,3398,1,4,1,3,1,4,…,2,2,1,1,2
7,6,4,0,1361,1,2,2,3,1,4,…,1,2,2,1,2
8,18,4,3,1098,1,1,4,2,1,4,…,2,1,1,1,1


## Seleccionar registros

Empezamos a familiarizarnos con la variable f que pertenece al módulo datatable. Esta variable es un mecanismo de datatable que nos permite referenciar a elementos del data table frame.

In [7]:
df4 = dt_df[dt.f['Duration of Credit (month)'] == 12, :]
df4.view()

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,12,2,9,841,2,4,2,2,1,4,…,1,2,1,1,1
1,12,4,0,2122,1,3,3,3,1,2,…,2,2,2,1,2
2,12,4,0,2171,1,3,4,3,1,4,…,2,2,1,1,2
3,12,4,0,3124,1,2,1,3,1,3,…,2,2,2,1,1
4,12,4,4,1424,1,4,4,3,1,3,…,1,3,1,1,1
5,12,2,2,652,1,5,4,2,1,4,…,1,3,1,1,1
6,12,4,0,1495,1,5,4,3,1,1,…,2,2,2,1,1
7,12,4,3,1934,1,5,2,3,1,2,…,2,3,1,1,1
8,12,4,5,996,5,4,4,2,1,4,…,2,3,1,1,1


Registros que cumplan una condición.

In [8]:
df4 = dt_df[dt.f['Duration of Credit (month)'] != 12, :]
df4.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,18,4,2,1049,1,2,4,2,1,4,…,1,3,1,1,1
1,9,4,0,2799,1,3,2,3,1,2,…,2,3,2,1,1
2,10,4,0,2241,1,2,1,3,1,3,…,2,2,2,1,2
3,8,4,0,3398,1,4,1,3,1,4,…,2,2,1,1,2
4,6,4,0,1361,1,2,2,3,1,4,…,1,2,2,1,2


Las condiciones complejas requieren el empleo de paréntesis. 

In [9]:
df5 = dt_df[(dt.f['Duration of Credit (month)'] == 12) & (dt.f['Credit Amount']<=1500), :]
df5.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,12,2,9,841,2,4,2,2,1,4,…,1,2,1,1,1
1,12,4,4,1424,1,4,4,3,1,3,…,1,3,1,1,1
2,12,2,2,652,1,5,4,2,1,4,…,1,3,1,1,1
3,12,4,0,1495,1,5,4,3,1,1,…,2,2,2,1,1
4,12,4,5,996,5,4,4,2,1,4,…,2,3,1,1,1


En este caso las variables tienen caracteres especiales en su definición por ello el uso de corchetes y comillas, sin embargo, la notación punto simplifica este tipo de trabajo:

In [10]:
df5 = dt_df[(dt.f['Duration of Credit (month)'] != 12) & (dt.f.Occupation == 3), :]
df5.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,18,4,2,1049,1,2,4,2,1,4,…,1,3,1,1,1
1,9,4,0,2799,1,3,2,3,1,2,…,2,3,2,1,1
2,11,4,0,3905,1,3,2,3,1,2,…,2,3,2,1,1
3,30,4,1,6187,2,4,1,4,1,4,…,2,3,1,1,1
4,6,4,3,1957,1,4,1,2,1,4,…,1,3,1,1,1


In [11]:
df6 = dt_df[(dt.f.Purpose != 0) | (dt.f.Occupation == 3), :]
df6.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪
0,18,4,2,1049,1,2,4,2,1,4,…,1,3,1,1,1
1,9,4,0,2799,1,3,2,3,1,2,…,2,3,2,1,1
2,12,2,9,841,2,4,2,2,1,4,…,1,2,1,1,1
3,18,4,3,1098,1,1,4,2,1,4,…,2,1,1,1,1
4,24,2,3,3758,3,1,1,2,1,4,…,1,1,1,1,1


## Creación de nuevas variables con Python

Creamos una distancia a la media del importe del crédito:

In [12]:
media = dt_df['Credit Amount'].mean()

In [13]:
dt_df['dist_media'] = dt_df[:,  dt.f['Credit Amount']/media]

In [14]:
dt_df.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,Occupation,No of dependents,Telephone,Foreign Worker,dist_media
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,18,4,2,1049,1,2,4,2,1,4,…,3,1,1,1,0.320673
1,9,4,0,2799,1,3,2,3,1,2,…,3,2,1,1,0.855637
2,12,2,9,841,2,4,2,2,1,4,…,2,1,1,1,0.257088
3,12,4,0,2122,1,3,3,3,1,2,…,2,2,1,2,0.648682
4,12,4,0,2171,1,3,4,3,1,4,…,2,1,1,2,0.663661


In [15]:
dt_df['dist_media'].mean()

,dist_media
,▪▪▪▪▪▪▪▪
0,1


En datatable la función ifelse nos permite crear condiciones:

In [16]:
media = dt.mean(dt_df['Credit Amount'])
media

Expr:mean(<Frame#7fb04c680690 1000x1>; )

Observemos que en este caso no hemos creado un data frame.

In [17]:
dt_df['Mayor que la media'] = dt.ifelse( dt_df['Credit Amount'] > media , 
                                        "Por encima de la media", "Por debajo de la media")

## Sumarizar datos

La forma más rápida y directa

In [18]:
dt_df[:, dt.count(dt.f.Occupation), dt.by(dt.f['Mayor que la media'])]

,Mayor que la media,Occupation
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,Por debajo de la media,658
1,Por encima de la media,342


Múltiples columnas.

In [26]:
resumen = dt_df[:, dt.mean(dt.f['Credit Amount']), dt.by(dt.f.Purpose,dt.f['Mayor que la media'])]
resumen.head(5)

,Purpose,Mayor que la media,Credit Amount
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,Por debajo de la media,1569.41
1,0,Por encima de la media,6634.45
2,1,Por debajo de la media,2363.53
3,1,Por encima de la media,6725.35
4,2,Por debajo de la media,1969.86


### Renombramos la columna

In [28]:
resumen.names = ['Media' if x == 'Credit Amount' else x for x in resumen.names]
resumen.head(5)

,Purpose,Mayor que la media,Media
,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,Por debajo de la media,1569.41
1,0,Por encima de la media,6634.45
2,1,Por debajo de la media,2363.53
3,1,Por encima de la media,6725.35
4,2,Por debajo de la media,1969.86
5,2,Por encima de la media,5072.66
6,3,Por debajo de la media,1676.06
7,3,Por encima de la media,5663
8,4,Por debajo de la media,1271.45


## Ordenar datos

Orden ascendente

In [32]:
orden = dt_df.sort('Credit Amount')
orden.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of dependents,Telephone,Foreign Worker,dist_media,Mayor que la media
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,6,4,0,250,4,3,2,2,1,2,…,1,1,1,0.0764234,Por debajo de la media
1,9,2,0,276,1,3,4,4,1,4,…,1,1,1,0.0843715,Por debajo de la media
2,6,4,3,338,3,5,4,3,1,4,…,1,1,1,0.103324,Por debajo de la media
3,12,1,8,339,1,5,4,4,1,1,…,1,1,1,0.10363,Por debajo de la media
4,6,2,4,343,1,2,4,2,1,1,…,1,1,1,0.104853,Por debajo de la media


Orden descendente

In [33]:
orden = dt_df[::-1,:, dt.sort(dt.f['Credit Amount'])]
orden.head(5)

,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,Duration in Current address,…,No of dependents,Telephone,Foreign Worker,dist_media,Mayor que la media
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪
0,48,0,10,18424,1,3,1,2,1,2,…,1,2,2,5.6321,Por encima de la media
1,54,0,9,15945,1,2,3,3,1,4,…,1,2,1,4.87429,Por encima de la media
2,36,2,10,15857,1,1,2,1,2,3,…,1,1,1,4.84739,Por encima de la media
3,48,2,9,15672,1,3,2,3,1,2,…,1,2,1,4.79083,Por encima de la media
4,60,3,3,15653,1,4,2,3,1,4,…,1,2,1,4.78502,Por encima de la media
